In [1]:
import numpy as np
import pandas as pd
import os # used for navigating to image path
import csv

In [2]:
TILES_ASSIGNMENT = '../../data/tidy/conflict-split-images/tiles_assignment.csv'

In [11]:
def generate_index(path_to_tiles_assignment):
    """Serially labels all tiles by class and generates an index to data/tidy/conflict_split_images"""
    
    data = pd.read_csv(path_to_tiles_assignment)
    data = data.drop([0])
    
    df = pd.DataFrame(columns=['original', 'tile', 'classification'])
    
    conflict_counter    = 1
    no_conflict_counter = 1
    unknown_counter = 1
    counters = {}
        
    for row in data.itertuples():
        tile = row.Filename
        original_image = str(os.path.splitext(tile)).split('_')[0][2:]
        classification = row.Classification

        if classification.lower() == 'c':        
            save_label = 'conflict' + '-' + str(conflict_counter) + '.jpg'
            df.loc[row.Index] = [original_image] + [tile] + [save_label]
            conflict_counter += 1
        elif classification.lower() == 'n':
            save_label = 'no_conflict' + '-' + str(no_conflict_counter) + '.jpg'
            df.loc[row.Index] = [original_image] + [tile] + [save_label]
            no_conflict_counter += 1
        else:
            save_label = 'unknown' + '-' + str(unknown_counter) + '.jpg'
            df.loc[row.Index] = [original_image] + [tile] + [save_label]
            unknown_counter += 1
    
    df.to_csv('../../data/tidy/conflict-split-images/tile_index_mapping.csv', encoding='utf-8', index=False)
    
    counters['conflict'] = conflict_counter - 1
    counters['no_conflict'] = no_conflict_counter - 1
    counters['unknown'] = unknown_counter -1 
    
    print('Number of conflict tiles recorded:', conflict_counter - 1)    
    print('Number of no-conflict tiles recorded:', no_conflict_counter - 1)
    print('Number of unknown tiles:', unknown_counter - 1)
        
    return df, counters

In [12]:
df, c = generate_index(TILES_ASSIGNMENT)

Number of conflict tiles recorded: 40
Number of no-conflict tiles recorded: 41
Number of unknown tiles: 0


In [13]:
df

,original,tile,classification
1,improbable-1,improbable-1_0_0.jpg,no_conflict-1.jpg
2,improbable-1,improbable-1_0_336.jpg,conflict-1.jpg
3,improbable-1,improbable-1_0_672.jpg,no_conflict-2.jpg
4,improbable-1,improbable-1_0_1008.jpg,conflict-2.jpg
5,improbable-1,improbable-1_0_1344.jpg,no_conflict-3.jpg
...,...,...,...
77,improbable-1,improbable-1_3584_1344.jpg,conflict-37.jpg
78,improbable-1,improbable-1_3584_1680.jpg,conflict-38.jpg
79,improbable-1,improbable-1_3584_2016.jpg,conflict-39.jpg
80,improbable-1,improbable-1_3584_2352.jpg,conflict-40.jpg
